# SOC Dashboard Aggregations

## Phase
Phase 5 — Dashboard Preparation

## Objective
Generate aggregated metrics from prioritized alerts to power SOC dashboard visualizations.


In [1]:
import pandas as pd
from pathlib import Path


In [2]:
PROJECT_ROOT = Path(r"D:\soc-dashboard-suite-main\soc-dashboard-suite-main")

INPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "alerts_scored_prioritized.csv"
DASHBOARD_DIR = PROJECT_ROOT / "data" / "dashboard"

DASHBOARD_DIR.mkdir(parents=True, exist_ok=True)

alerts_df = pd.read_csv(INPUT_PATH, parse_dates=["event_time"])
alerts_df.head()


,event_time,sender_email,sender_domain,recipient_email,user_role,domain_rarity,is_first_seen_day,time_behavior,severity,alert_reason,detection_id,source_rule_x,source_rule_y,risk_score,rule_count,priority
0,NaT,cramer@cadvision.com,cadvision.com,"john.zufferli@enron.com, demers.nicolas@enron....",admin,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK,DET_03_MULTI_SIGNAL_EMAIL_RISK,"DET_01_SUSPICIOUS_SENDER_DOMAIN,DET_02_HIGH_RI...",100,1,P1
1,NaT,mark.shea@bankofamerica.com,bankofamerica.com,"jesus.melendrez@enron.com, john.griffith@enron...",executive,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK,DET_03_MULTI_SIGNAL_EMAIL_RISK,"DET_01_SUSPICIOUS_SENDER_DOMAIN,DET_02_HIGH_RI...",100,1,P1
2,2000-09-07 02:17:00-07:00,tmcauliff@isda.org,isda.org,"arothrock@pattonboggs.com, csteffensen@isda.or...",executive,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK,DET_03_MULTI_SIGNAL_EMAIL_RISK,"DET_01_SUSPICIOUS_SENDER_DOMAIN,DET_02_HIGH_RI...",100,1,P1
3,NaT,penn_eric@smtpgate.salkeiz.k12.or.us,smtpgate.salkeiz.k12.or.us,"mark.guzman@enron.com, jones@mca-architects.co...",admin,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK,DET_03_MULTI_SIGNAL_EMAIL_RISK,"DET_01_SUSPICIOUS_SENDER_DOMAIN,DET_02_HIGH_RI...",100,1,P1
4,NaT,matt.hsu@interwoven.com,interwoven.com,mleslie@amgen.com,admin,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK,DET_03_MULTI_SIGNAL_EMAIL_RISK,"DET_01_SUSPICIOUS_SENDER_DOMAIN,DET_02_HIGH_RI...",100,1,P1


In [3]:
alerts_by_priority = alerts_df["priority"].value_counts().reset_index()
alerts_by_priority.columns = ["priority", "alert_count"]

alerts_by_priority.to_csv(DASHBOARD_DIR / "alerts_by_priority.csv", index=False)
alerts_by_priority


,priority,alert_count
0,P4,22032
1,P2,11513
2,P1,1839
3,P3,1249


In [4]:
alerts_df["date"] = alerts_df["event_time"].dt.date

alerts_over_time = alerts_df.groupby("date").size().reset_index(name="alert_count")
alerts_over_time.to_csv(DASHBOARD_DIR / "alerts_over_time.csv", index=False)

alerts_over_time.head()


,date,alert_count
0,1986-05-01,1
1,1999-04-05,1
2,1999-04-26,1
3,1999-04-27,1
4,1999-05-03,2


In [6]:
if "triggered_rules" not in alerts_df.columns:
    alerts_df["triggered_rules"] = alerts_df["detection_id"]


In [7]:
rule_counts = (
    alerts_df["triggered_rules"]
    .str.split(",")
    .explode()
    .value_counts()
    .reset_index()
)

rule_counts.columns = ["detection_rule", "alert_count"]

rule_counts.to_csv(DASHBOARD_DIR / "alerts_by_detection_rule.csv", index=False)
rule_counts


,detection_rule,alert_count
0,DET_01_SUSPICIOUS_SENDER_DOMAIN,23819
1,DET_02_HIGH_RISK_EMAIL_TIMING,10986
2,DET_03_MULTI_SIGNAL_EMAIL_RISK,1828


In [8]:
alerts_by_role = alerts_df["user_role"].value_counts().reset_index()
alerts_by_role.columns = ["user_role", "alert_count"]

alerts_by_role.to_csv(DASHBOARD_DIR / "alerts_by_user_role.csv", index=False)
alerts_by_role


,user_role,alert_count
0,normal,23281
1,admin,6965
2,executive,6387


In [9]:
top_risky_domains = (
    alerts_df["sender_domain"]
    .value_counts()
    .head(10)
    .reset_index()
)

top_risky_domains.columns = ["sender_domain", "alert_count"]

top_risky_domains.to_csv(DASHBOARD_DIR / "top_risky_domains.csv", index=False)
top_risky_domains


,sender_domain,alert_count
0,enron.com,10361
1,houston.rr.com,241
2,amazon.com,223
3,msn.com,197
4,brobeck.com,156
5,schwab.com,141
6,motleyfool.com,131
7,enform.com,130
8,pge.com,128
9,andrews-kurth.com,126


In [10]:
priority_rule_matrix = (
    alerts_df.assign(rule=alerts_df["triggered_rules"].str.split(","))
    .explode("rule")
    .pivot_table(index="priority", columns="rule", aggfunc="size", fill_value=0)
)

priority_rule_matrix.to_csv(DASHBOARD_DIR / "priority_vs_rule_matrix.csv")
priority_rule_matrix


rule,DET_01_SUSPICIOUS_SENDER_DOMAIN,DET_02_HIGH_RISK_EMAIL_TIMING,DET_03_MULTI_SIGNAL_EMAIL_RISK
priority,,,
P1,0,11,1828
P2,538,10975,0
P3,1249,0,0
P4,22032,0,0
